# 1. Set Up

In [1]:
# !spack find -lvp tfel
# !echo "$(spack find -p tfel | awk '/\/mofem_install\// {print $NF "/lib"}')"
# %env LD_LIBRARY_PATH=/mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-mvfpqw7u4c23su7hj7g4leuwmykrjmcx/lib

In [2]:
import math
import os
import re
import sys
import time
import json
from pathlib import Path
import subprocess
import zipfile
import pydantic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyvista as pv
import gmsh
# from scipy import optimize
# from scipy.optimize import curve_fit, least_squares

sys.path.append('/mofem_install/jupyter/thomas/mfront_example_test/src')

import setup
import core
import custom_models as cm
import utils as ut
import plotting
    


# 2. Simulation Parameters

In [ ]:
#in MPa
soil_model = cm.PropertyTypeEnum.le
# soil_model = cm.PropertyTypeEnum.cam_clay
def initialize_parameters() -> cm.AttrDict:
    params = cm.AttrDict()
    params.interface = False
    params.global_default_model = cm.PropertyTypeEnum.le
    params.pile_manager = cm.PileManager(x=0, y=0, z=0, dx=0, dy=0, R=1, r=0.975,
                                         stickup_height = 10,
                                         embedded_depth = -10.5,
                                preferred_model= cm.PropertyTypeEnum.le,
                                props = {
                                    cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=200000, poisson_ratio=0.3),
                                },
                                interface = params.interface,
                                )

    soil_layer_1 = cm.SoilLayer(
        depth = -2,
        preferred_model= params.global_default_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=96, poisson_ratio=0.499),
            cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=96, poisson_ratio=0.499,HardeningSlope = 10, YieldStress = 5),
            cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=96, poisson_ratio=0.499, phi=np.radians(27), c=0.1, v=np.radians(27)),
            cm.PropertyTypeEnum.mcc: cm.CamClayProperties(),
            }, 
            )
    soil_layer_2 = cm.SoilLayer(
        depth = -1.4,
        preferred_model= params.global_default_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=182.1, poisson_ratio=0.499),
            cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=182.1, poisson_ratio=0.499,HardeningSlope = 10, YieldStress = 5),
            cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=182.1, poisson_ratio=0.499, phi=np.radians(27), c=0.1, v=np.radians(27)),
            cm.PropertyTypeEnum.mcc: cm.CamClayProperties(),
            }, 
            )
    soil_layer_3 = cm.SoilLayer(
        depth = -7.1,
        preferred_model= params.global_default_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=351.3, poisson_ratio=0.499),
            cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=351.3, poisson_ratio=0.499,HardeningSlope = 10, YieldStress = 5),
            cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=351.3, poisson_ratio=0.499, phi=np.radians(27), c=0.1, v=np.radians(27)),
            cm.PropertyTypeEnum.mcc: cm.CamClayProperties(),
                 }, 
            )
    soil_layer_4 = cm.SoilLayer(
        depth = -29.5,
        preferred_model= params.global_default_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=668.4, poisson_ratio=0.499),
            cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=668.4, poisson_ratio=0.499,HardeningSlope = 10, YieldStress = 5),
            cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=668.4, poisson_ratio=0.499, phi=np.radians(27), c=0.1, v=np.radians(27)),
            cm.PropertyTypeEnum.mcc: cm.CamClayProperties(),
            }, 
            )
    params.interface_manager = cm.InterfaceManager(
        preferred_model = cm.PropertyTypeEnum.vM,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=96, poisson_ratio=0.3),
            cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=96, poisson_ratio=0.499,HardeningSlope = 5000, YieldStress = 20),
            cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=668.4, poisson_ratio=0.499, phi=np.radians(27), c=0, v=np.radians(27)),
            },
            )
    # params.prescribed_force = cm.ForceBoundaryCondition(fx=0,fy=0,fz=0)
    params.prescribed_disp = cm.SurfaceBoundaryCondition(disp_ux=1, disp_uy=0 ,disp_uz=0)
    params.box_manager = cm.BoxManager(x=-80, y=-80, z=0, dx=160, dy=80,
        layers=[
        soil_layer_1,
        soil_layer_2,
        soil_layer_3,
        soil_layer_4,   
        ],
        far_field_size=5,
        near_field_dist=5,
        near_field_size=0.05,
    )
    params.cylinder_manager = cm.CylinderManager(
        x=0, 
        y=0, 
        z=0, 
        dx=0, 
        dy=0, 
        r=80,
        pile_R = params.pile_manager.R,
        pile_r = params.pile_manager.r,
        layers=[
        soil_layer_1,
        soil_layer_2,
        soil_layer_3,
        soil_layer_4,   
        ],
        far_field_size=5,
        near_field_dist=5,
        near_field_size=0.05,
        radial_progression = 1.3,
    )
    params.mesh_radial_divisions = 3
    params.nproc = 8 # number of processors/cores used
    params.order = 2 #order of approximation functions
    params.dim = 3
    
    params.final_time = 1 # [s]
    params.time_step = 1 / 5 # [s]
    params.case_name = "pile_manual"
    # params.base = "hex"
    params.wk_dir = Path(f"/mofem_install/jupyter/thomas/mfront_example_test")
    params.read_med_exe = "/mofem_install/jupyter/thomas/um_view/bin/read_med"
    params.h5m_to_vtk_converter = "/mofem_install/jupyter/thomas/um_view/bin/convert.py"
    params.partition_exe = "/mofem_install/jupyter/thomas/um_view/bin/mofem_part"
    params.exe = f"/mofem_install/jupyter/thomas/um_view/tutorials/adv-1/contact_3d"
    params.postprocessing_script = "/mofem_install/jupyter/thomas/mfront_example_test/src/pile_postprocessing.py"
    params.mode = "auto" # see 4. to see difference between auto and manual

    return params

In [4]:
params = initialize_parameters()

# 3. Log paths and meta

In [5]:
params = setup.setup(params)

Simulation #223 for the day.


# 4. Generate the mesh

In [6]:
core.generate_mesh(params)

╭──────────────────────────╮
│  (MANUAL) DRAWING MESH   │
╰──────────────────────────╯
Done, taken Wall Time: 12.86 seconds, CPU Time: 12.80 seconds
╭───────────────────────────────────────────╮
│  (MANUAL) ADDING PHYSICAL GROUPS TO MESH  │
╰───────────────────────────────────────────╯
Done, taken Wall Time: 0.00 seconds, CPU Time: 0.00 seconds
╭──────────────────────╮
│  CHECKING BLOCK IDS  │
╰──────────────────────╯
{'pile_manual': 1, 'CYLINDER': 2, 'FIX_ALL': 3, 'FIX_X_0': 4, 'FIX_X_1': 5, 'FIX_Y_0': 6, 'FIX_Z_0': 7, 'SOIL_LAYER_0': 8, 'SOIL_LAYER_1': 9, 'SOIL_LAYER_2': 10, 'SOIL_LAYER_3': 11}
Done, taken Wall Time: 0.46 seconds, CPU Time: 0.00 seconds
╭───────────────────────────╮
│  GENERATING CONFIG FILES  │
╰───────────────────────────╯
[PhysicalGroup(dim=3, tags=[1, 2, 3], name='SOIL_LAYER_0', meshnet_id=8, group_type=<PhysicalGroupType.MATERIAL: 1>, preferred_model=<PropertyTypeEnum.le: 'LinearElasticity'>, props={<PropertyTypeEnum.le: 'LinearElasticity'>: ElasticProperties(yo

[0] <inform> MoFEM version 0.14.0 (MOAB 5.5.0 Petsc Release Version 3.16.6, Mar 30, 2022 )
[0] <inform> git commit id 693a01d8ad1e79f10b1dc4f1dd785f311161b1d1
[0] <inform> Local time: 2024-11-14 23:43:56
[0] <inform> UTC time: 2024-11-14 23:43:56
[0] <inform> [MED] Reading MED file V4.1.0 using MED library V4.1.0
[0] <inform> [MED] Check mesh pile_manual nsteps 1
[0] <inform> [MED] Reading mesh pile_manual nsteps 1
[0] <inform> [MED] Read number of nodes 115953
[0] <inform> [MED] Reading elements 20310 of type Tri number of nodes 3
[0] <inform> [MED] Reading elements 595137 of type Tet number of nodes 4
[0] <inform> [read_med] Print all meshsets (old and added from meshsets configurational file)
[0] <inform> [read_med] meshset 12682136550675316737 type BLOCKSET UNKNOWNNAME msId 1 name pile_manual
[0] <inform> [read_med] meshset 12682136550675316738 type BLOCKSET UNKNOWNNAME msId 2 name CYLINDER                       
[0] <inform> [read_med] meshset 12682136550675316739 type BLOCKSET UN

Done, taken Wall Time: 0.48 seconds, CPU Time: 0.00 seconds
╭─────────────────────────────────────╮
│  PARTITIONING MESH with mofem_part  │
╰─────────────────────────────────────╯
Done, taken Wall Time: 137.23 seconds, CPU Time: 0.00 seconds


{'interface': False,
 'global_default_model': <PropertyTypeEnum.le: 'LinearElasticity'>,
 'pile_manager': PileManager(x=0.0, y=0.0, z=0.0, dx=0.0, dy=0.0, R=1.0, r=0.975, stickup_height=10.0, embedded_depth=-10.5, interface=False, preferred_model='LinearElasticity', props={'LinearElasticity': ElasticProperties(youngs_modulus=200000.0, poisson_ratio=0.3)}),
 'interface_manager': InterfaceManager(preferred_model=<PropertyTypeEnum.vM: 'VMSimo'>, props={<PropertyTypeEnum.le: 'LinearElasticity'>: ElasticProperties(youngs_modulus=96.0, poisson_ratio=0.3), <PropertyTypeEnum.vM: 'VMSimo'>: VonMisesProperties(youngs_modulus=96.0, poisson_ratio=0.499, HardeningSlope=5000.0, YieldStress=20.0), <PropertyTypeEnum.dp: 'DruckerPragerSimple'>: DruckerPragerProperties(youngs_modulus=668.4, poisson_ratio=0.499, phi=0.47123889803846897, c=0.0, v=0.47123889803846897)}),
 'prescribed_disp': SurfaceBoundaryCondition(disp_ux=1.0, disp_uy=0.0, disp_uz=0.0),
 'box_manager': BoxManager(x=-80.0, y=-80.0, z=0.0, 

# 5. Running the analysis and export to .vtk file format

In [ ]:
core.mofem_compute(params)

╭─────────────╮
│  COMPUTING  │
╰─────────────╯
Command:
['bash', '-c', 'export OMPI_MCA_btl_vader_single_copy_mechanism=none && nice -n 10 mpirun --oversubscribe --allow-run-as-root -np 8 /mofem_install/jupyter/thomas/um_view/tutorials/adv-1/contact_3d -file_name /mofem_install/jupyter/thomas/mfront_example_test/simulations/pile_manual_day_89_sim_223_20241114_234342_le/pile_manual_day_89_sim_223_le_8p.h5m -sdf_file /mofem_install/jupyter/thomas/mfront_example_test/src/sdf.py -order 2 -contact_order 0 -sigma_order 0  -ts_dt 0.2 -ts_max_time 1 -mi_lib_path_108 /mofem_install/jupyter/thomas/mfront_interface/src/libBehaviour.so -mi_block_108 LinearElasticity -mi_param_108_0 96.0 -mi_param_108_1 0.3 -mi_param_108_2 0 -mi_param_108_3 0 -mi_param_108_4 0 -mi_param_108_5 0  -mi_lib_path_109 /mofem_install/jupyter/thomas/mfront_interface/src/libBehaviour.so -mi_block_109 LinearElasticity -mi_param_109_0 182.1 -mi_param_109_1 0.3 -mi_param_109_2 0 -mi_param_109_3 0 -mi_param_109_4 0 -mi_param_1

[0] <inform> MoFEM version 0.14.0 (MOAB 5.5.0 Petsc Release Version 3.16.6, Mar 30, 2022 )
[0] <inform> git commit id 693a01d8ad1e79f10b1dc4f1dd785f311161b1d1
[0] <inform> Local time: 2024-11-14 23:46:14
[0] <inform> UTC time: 2024-11-14 23:46:14
[0] <warning> [MeshsetMng] broadcasted vertex 15647 has negative id
[0] <warning> [MeshsetMng] broadcasted vertex 15647 has negative id
[0] <inform> [MeshsetMng] meshset in database meshset 12682136550675316738 type BLOCKSET UNKNOWNNAME msId 1 name pile_manual
[0] <inform> [MeshsetMng] meshset in database meshset 12682136550675316739 type BLOCKSET UNKNOWNNAME msId 2 name CYLINDER                       
[0] <inform> [MeshsetMng] meshset in database meshset 12682136550675316761 type BLOCKSET UNKNOWNNAME msId 3 name FIX_ALL                        
[0] <inform> [MeshsetMng] meshset in database meshset 12682136550675316759 type BLOCKSET UNKNOWNNAME msId 4 name FIX_X_0                        
[0] <inform> [MeshsetMng] meshset in database meshset 126

In [ ]:
core.export_to_vtk(params)

# 6. Visualize final time step .vtk file

In [ ]:
import pyvista as pv
pv.set_plot_theme("document")

from pyvirtualdisplay import Display
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()
vtk_files = subprocess.run(f"ls -c1 {params.data_dir}/*.vtk | sort -V", shell=True, text=True, capture_output=True)
if vtk_files.returncode == 0:
    files = [vtk_file for vtk_file in vtk_files.stdout.splitlines()]
    final_file = files[-1]
    mesh = pv.read(final_file)
    mesh=mesh.shrink(0.95) 
    warp_factor = 1.0
    # mesh = mesh.warp_by_vector(vectors="U", factor = warp_factor)
    # show_field = "STRESS"
    show_field = "STRAIN" # U: displacement
    # show_field = "STRAIN" # U: displacement
    # print(mesh.point_data)
    # if mesh.point_data[show_field].shape[1] > 3:
        # cmap = "Spectral"
    p = pv.Plotter(notebook=True)
    p.add_mesh(mesh, scalars=show_field)
    # p.camera_position = [(-10, 0, 10), (0.0, 0.0, 0.0), (0.0, 1.0, 0.0)]
    p.camera_position = 'iso'
    p.show(jupyter_backend='ipygany')

In [ ]:
sys.exit()

# 7. Extract data from .vtk files 
Tools:
pvpython

In [ ]:
if params:
    postprocessing_script = params.postprocessing_script
    data_dir = params.data_dir
else:
    postprocessing_script = "/mofem_install/jupyter/thomas/mfront_example_test/src/pile_postprocessing.py"
    data_dir = "/mofem_install/jupyter/thomas/mfront_example_test/simulations/day_39_sim_3_2024_09_25_18_50_27"
original_pythonpath = os.environ.get("PYTHONPATH", "")
os.environ["PYTHONPATH"] = ""
try:
    !/mofem_install/jupyter/thomas/ParaView-5.13.1-MPI-Linux-Python3.10-x86_64/bin/pvpython {postprocessing_script} {data_dir}
except Exception as e:
    print(e)
finally:
    os.environ["PYTHONPATH"] = original_pythonpath

In [ ]:
paraview_path = "/mofem_install/jupyter/thomas/ParaView-5.13.1-MPI-Linux-Python3.10-x86_64/bin/pvpython"

original_pythonpath = os.environ.get("PYTHONPATH", "")
os.environ["PYTHONPATH"] = ""
# Properly format the command in Python
# Construct the command as a list of strings

@ut.track_time("PULLING A SELECTED POINT OVER TIME WITH pvpython")
def to_csv():
    command = [
        paraview_path,
        "/mofem_install/jupyter/thomas/mfront_example_test/src/pvpython_scripts/point_to_time_csv.py",
        params.vtk_dir,
        params.point_to_time_filepath,
        str(int(-1)),
        str(0),
        str(10),
    ]
    # Run the command using subprocess
    try:
        process = subprocess.run(command, check=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        print("An error occurred:", e.stderr.decode())
    finally:
    # Restore the original PYTHONPATH
        os.environ["PYTHONPATH"] = original_pythonpath
        
to_csv()
df = pd.read_csv(params.point_to_time_filepath)
strain_magnitude = np.array(df['avg(STRAIN (Magnitude))'])
color_max = strain_magnitude.max()
color_min = strain_magnitude.min()

@ut.track_time("ANIMATING OVER TIME WITH pvpython")
def animate():
    command = [
        paraview_path,
        "/mofem_install/jupyter/thomas/mfront_example_test/src/pvpython_scripts/test_3D_animate.py",
        params.vtk_dir,
        params.strain_animation_filepath_png,
        str(color_min),
        str(color_max),
    ]
    # Run the command using subprocess
    try:
        process = subprocess.run(command, check=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        print("An error occurred:", e.stderr.decode())
    finally:
    # Restore the original PYTHONPATH
        os.environ["PYTHONPATH"] = original_pythonpath

#Stitch together animations
@ut.track_time("STITCHING .pngs TOGETHER with ffmpeg")
def create_mp4_from_png_sequence(animation_filepath_png_ffmpeg_regex, animation_filepath_mp4, framerate=40):
    # Build the ffmpeg command
    ffmpeg_command = [
        '/mofem_install/jupyter/thomas/ffmpeg-7.0.2-amd64-static/ffmpeg',
        '-framerate', str(framerate),  # Set input framerate
        '-y',
        '-i', animation_filepath_png_ffmpeg_regex,  # Input image sequence (with regex pattern)
        '-c:v', 'libx264',  # Set video codec to libx264
        '-pix_fmt', 'yuv420p',  # Set pixel format for compatibility
        '-loglevel', 'warning',
        animation_filepath_mp4  # Output .mp4 file
    ]

    # Run the ffmpeg command as a subprocess
    try:
        subprocess.run(ffmpeg_command, check=True, capture_output=True)
        print(f"MP4 video created successfully: {animation_filepath_mp4}")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during ffmpeg execution: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        
animate()
create_mp4_from_png_sequence(params.strain_animation_filepath_png_ffmpeg_regex, params.strain_animation_filepath_mp4, framerate=40)


# 8. Plotting
Tools:
Matplotlib

## AT A SINGLE POINT

In [ ]:
import sys

import matplotlib
import numpy as np
sys.path.append('/mofem_install/jupyter/thomas/mfront_example_test/src')

import plotting
import calculations as calc


matplotlib.rc('figure', figsize=(7, 7))
import pandas as pd
df = pd.read_csv(params.point_to_time_filepath)
sig_xx = np.array(df['avg(STRESS (0))'])
sig_xy = np.array(df['avg(STRESS (1))'])
sig_xz = np.array(df['avg(STRESS (2))'])
sig_yy = np.array(df['avg(STRESS (4))'])
sig_yz = np.array(df['avg(STRESS (5))'])
sig_zz = np.array(df['avg(STRESS (8))'])

e_xx = np.array(df['avg(STRAIN (0))'])
e_xy = np.array(df['avg(STRAIN (1))'])
e_xz = np.array(df['avg(STRAIN (2))'])
e_yy = np.array(df['avg(STRAIN (4))'])
e_yz = np.array(df['avg(STRAIN (5))'])
e_zz = np.array(df['avg(STRAIN (8))'])

sig_1, sig_2, sig_3 = calc.calculate_principal_stresses(sig_xx, sig_yy, sig_zz, sig_xy, sig_xz, sig_yz)
p = calc.calculate_p(sig_1, sig_2, sig_3)
J_2 = calc.calculate_J2(sig_1, sig_2, sig_3)
J  = np.sqrt(J_2)
tau_oct = np.sqrt(2 * J_2)
sig_eq = np.sqrt(3 * J_2)
e_v, e_d = calc.calculate_volumetric_and_deviatoric_strain(e_xx, e_yy, e_zz, e_xy, e_xz, e_yz)

def plot_x_ys(x_array: list, y_arrays, labels: list, cutoffs=None, x_label="", y_label="", title="", save_as: str = None):
    data = []
    for i in range(len(y_arrays)):
        data.append((x_array, y_arrays[i], labels[i], 'g', None))
    return plotting.create_plot(data, x_label, y_label, title, save_as)
print(f"exx: {e_xx[-1]}")
print(f"eyy: {e_yy[-1]}")
print(f"ezz: {e_zz[-1]}")
print(f"sig1: {sig_1[-1]}")
print(f"sig2: {sig_2[-1]}")
print(f"sig3: {sig_3[-1]}")
print(f"p: {p[-1]}")
print(f"J2: {J_2[-1]}")
print(f"J: {J[-1]}")
print(f"sig_eq: {sig_eq[-1]}")
label = f"""Drucker-Prager (FEA)
E = {params.tester.props[params.global_default_model].youngs_modulus}
nu = {params.tester.props[params.global_default_model].poisson_ratio}
c = {params.tester.props[params.global_default_model].c}
phi = {np.degrees(params.tester.props[params.global_default_model].phi)}
v = {np.degrees(params.tester.props[params.global_default_model].v)}
"""
plot_x_ys(p, [sig_eq], labels=[label], x_label='Hydrostatic stress $p$', y_label='Equivalent stress $sig_{eq}$', title='Equivalent Stress vs Hydrostatic stress', save_as = f"{params.graph_dir}/111_sigeq_p.png")

plot_x_ys(e_zz, [sig_eq], labels=[label], x_label='Axial Strain $\epsilon_{zz}$', y_label='Equivalent Stress $\sigma_{eq}$', title='Equivalent Stress $\sigma_{eq}$ vs Axial Strain $\epsilon_{zz}$', save_as = f"{params.graph_dir}/201_sigeq_eyy.png")
plot_x_ys(e_zz, [sig_zz], labels=["test1"], x_label='Axial Strain $\epsilon_{zz}$', y_label='Stress $\sigma_{zz}$', title='Stress $\sigma_{zz}$ vs Axial Strain $\epsilon_{zz}$', save_as = f"{params.graph_dir}/201_sigeq_ezz.png")
plot_x_ys(e_zz, [sig_yy], labels=["test1"], x_label='Axial Strain $\epsilon_{zz}$', y_label='Stress $\sigma_{yy}$', title='Stress $\sigma_{yy}$ vs Axial Strain $\epsilon_{zz}$', save_as = f"{params.graph_dir}/201_sigeq_eyy.png")
plot_x_ys(e_zz, [sig_xx], labels=["test1"], x_label='Axial Strain $\epsilon_{zz}$', y_label='Stress $\sigma_{xx}$', title='Stress $\sigma_{xx}$ vs Axial Strain $\epsilon_{zz}$', save_as = f"{params.graph_dir}/201_sigeq_exx.png")

plot_x_ys(e_zz[1:], [sig_eq[1:]/p[1:]], labels=["test1"], x_label='Axial Strain $\epsilon_{yy}$', y_label='Stress ratio $q/p$', title='Stress ratio vs Axial Strain', save_as = f"{params.graph_dir}/211_sigeq_div_p_eyy.png")

plot_x_ys(e_d, [e_v], labels=["volumetric - axial"], x_label='Axial strain $\epsilon_{zz}$', y_label='Volumetric strain $\epsilon^v$', title='Volumetric strain vs Axial strain', save_as = f"{params.graph_dir}/302_ev_ed.png")
plot_x_ys(e_d, [e_d], labels=["Deviatoric - axial"], x_label='Axial strain $\epsilon_{zz}$', y_label='Deviatoric strain $\epsilon^v$', title='Deviatoric strain vs Axial strain', save_as = f"{params.graph_dir}/303_ev_ed.png")
plot_x_ys(e_d, [e_v], labels=["FEA"], x_label='Deviatoric strain $\epsilon^d$', y_label='Volumetric strain $\epsilon^v$', title='Volumetric strain vs Deviatoric strain', save_as = f"{params.graph_dir}/304_ev_ed.png")


In [ ]:
# import sys
# sys.path.append('/mofem_install/jupyter/thomas/mfront_example_test/src')
# import plotting
# import custom_models as cm
# data_dir = r"/mofem_install/jupyter/thomas/mfront_example_test/simulations/auto_LinearElasticity_d39_8_2024_09_25_22_16_19"
# mode = params.mode if params else "auto"
# # data_dir = r"/mofem_install/jupyter/thomas/mfront_example_test/simulations/manual_LinearElasticity_d39_7_2024_09_25_20_58_41"
# # mode = "manual"
# soil_model = cm.PropertyTypeEnum.elastic
# suptitle = f"Mesh mode: {mode} | Behaviour: {soil_model.value}"

# time_plot_config = plotting.PlotConfig(
#     csv_files = {
#         "1.2,0,0 (Compression)": f'{data_dir}/dis_to_time_compression_mudline_1.2.csv', 
#         "1.1,0,0 (Compression)": f'{data_dir}/dis_to_time_compression_mudline_1.1.csv', 
#         "1.0,0,0 (Compression)": f'{data_dir}/dis_to_time_compression_mudline_1.0.csv', 
#      }, 
#     output_dir = data_dir,
#     suptitle = suptitle,
# )

# depth_plot_config = plotting.PlotConfig(
#     csv_files = {
#         "1.2,0,0 (Compression)": f'{data_dir}/dis_to_depth_compression_x_1.2.csv', 
#         "1.1,0,0 (Compression)": f'{data_dir}/dis_to_depth_compression_x_1.1.csv', 
#         "1.0,0,0 (Compression)": f'{data_dir}/dis_to_depth_compression_x_1.csv', 
#      }, 
#     output_dir = data_dir,
#     suptitle = suptitle,
# )

# against_time_plotter = plotting.TimePlotter(time_plot_config)
# against_depth_plotter = plotting.DepthPlotter(depth_plot_config)

# sigvM_e = against_time_plotter.sigvM_e(save_as="sigvM_e.png")
# J_p = against_time_plotter.J_p(save_as="J_p.png")

# dispx_z = against_depth_plotter.plot_displacement_vs_depth(save_as="dispx_z.png")
# sigxx_z = against_depth_plotter.sigxx_z(save_as="sigxx_z.png")
# sigvM_z = against_depth_plotter.plot_von_mises_vs_depth(save_as="sigvM_z.png")
# E_z = against_depth_plotter.E_z(save_as="E_z.png")

In [ ]:
# from IPython.display import display, HTML

# # Time-related images with a common title
# display(HTML('<h3 style="text-align:left">Across Time Steps</h3>'))

# image_files_time = [sigvM_e, J_p]
# images_html_time = ''.join([
#     f'<img src="{img}" style="display:inline;margin:1px" width="300"/>'
#     for img in image_files_time
# ])
# display(HTML(images_html_time))

# # Depth-related images with a common title
# display(HTML('<h3 style="text-align:elft">At the Final Time Step</h3>'))

# image_files_depth = [dispx_z, sigxx_z, sigvM_z, E_z]
# images_html_depth = ''.join([
#     f'<img src="{img}" style="display:inline;margin:1px" width="300"/>'
#     for img in image_files_depth
# ])
# display(HTML(images_html_depth))